# Kapitola 13: A* algoritmus - Jak dat AI sesty smysl

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | Algoritmy a hledani</h2>
    <p style="color: white; margin: 10px 0;">Heuristiky a informovane prohledavani</p>
</div>

## Co se naucite v teto kapitole

1. **Heuristika** - Co je to a proc ji potrebujeme
2. **A* algoritmus** - Kombinace reality a intuice
3. **Prioritni fronta** - Efektivni vyber nejlepsiho uzlu
4. **Manhattanska vzdalenost** - Prakticky heuristika pro mrizku
5. **Vizualizace** - Jak A* hleda cestu v bludisti

---

In [ ]:
# Inicializace prostredi pro Google Colab
import sys
from datetime import datetime

print("=" * 60)
print("KAPITOLA 13: A* algoritmus")
print("=" * 60)
print(f"Python verze: {sys.version.split()[0]}")
print(f"Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

# Instalace knihoven
!pip install -q matplotlib numpy

print("\nProstredi pripraveno!")

---

## Motivace: Proc potrebujeme heuristiku?

V minule kapitole jsme videli:
- **BFS** - najde nejkratsi cestu, ale prozkouma HODNE uzlu
- **DFS** - rychly, ale nenajde nejkratsi cestu

**Co kdybychom meli to nejlepsi z obou svetu?**

A* algoritmus pouziva **heuristiku** - "sesty smysl", ktery napovida, kterym smerem je cil.

In [ ]:
# Priklad: Heuristika jako kompas

print("CO JE HEURISTIKA?")
print("=" * 50)
print("""
Heuristika = informovany odhad vzdalenosti do cile

Priklad: Hledate cestu z Prahy do Brna

    Praha ----?---- Brno
           ~200 km
           (vzdusnou carou)

Vzdusna cara je HEURISTIKA:
- Je to ODHAD, ne presna vzdalenost
- Skutecna cesta je VZDY delsi nebo stejna
- Pomaha nam rozhodnout, kterym smerem jit

Dulezite pravidlo:
    Heuristika NESMI precenit skutecnou vzdalenost!
    (vzdusna cara je vzdy <= skutecna cesta)
""")

---

## Manhattanska vzdalenost

Pro mrizku (grid) pouzivame **Manhattanskou vzdalenost**:

```
h(A, B) = |x_A - x_B| + |y_A - y_B|
```

Proc "Manhattanska"? V Manhattanu (NYC) jsou ulice v mrizce - muzete jit jen nahoru/dolu/vlevo/vpravo.

In [ ]:
def manhattan_distance(a, b):
    """
    Manhattanska vzdalenost mezi dvema body.
    
    Args:
        a: tuple (row, col) - prvni bod
        b: tuple (row, col) - druhy bod
    
    Returns:
        int - manhattanska vzdalenost
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Demonstrace
print("MANHATTANSKA VZDALENOST")
print("=" * 50)

# Vizualizace
print("""
Mrizka 5x5:

    0   1   2   3   4
  +---+---+---+---+---+
0 | S |   |   |   |   |
  +---+---+---+---+---+
1 |   |   |   |   |   |
  +---+---+---+---+---+
2 |   |   |   |   |   |
  +---+---+---+---+---+
3 |   |   |   |   | C |
  +---+---+---+---+---+

S = Start (0,0)
C = Cil (3,4)
""")

start = (0, 0)
cil = (3, 4)

h = manhattan_distance(start, cil)
print(f"Manhattanska vzdalenost: |{start[0]}-{cil[0]}| + |{start[1]}-{cil[1]}| = {h}")
print(f"\nTo znamena: minimalne {h} kroku (bez prekazek)")

---

## A* algoritmus: f(n) = g(n) + h(n)

A* kombinuje DVE hodnoty pri rozhodovani:

- **g(n)** = skutecna cena cesty od STARTU do uzlu n (uz usla vzdalenost)
- **h(n)** = heuristika - ODHAD ceny od n do CILE
- **f(n)** = g(n) + h(n) = celkova odhadovana cena

A* vzdy prozkouma uzel s **nejnizsim f(n)**.

In [ ]:
import heapq

def a_star(grid, start, goal):
    """
    A* algoritmus pro hledani nejkratsi cesty v mrizce.
    
    Args:
        grid: 2D list - mrizka (0=volno, 1=zed)
        start: tuple (row, col)
        goal: tuple (row, col)
    
    Returns:
        (path, explored) - cesta a prozkoumane uzly
    """
    rows = len(grid)
    cols = len(grid[0])
    
    # Smery pohybu: nahoru, dolu, vlevo, vpravo
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # Prioritni fronta: (f_score, g_score, pozice, cesta)
    # Pouzivame g_score jako druhy klic pro stabilni razeni
    open_set = []
    h_start = manhattan_distance(start, goal)
    heapq.heappush(open_set, (h_start, 0, start, [start]))
    
    # Slovnik pro sledovani nejlepsiho g_score pro kazdy uzel
    g_scores = {start: 0}
    
    # Pro vizualizaci: poradi prozkoumavani
    explored = []
    closed_set = set()
    
    while open_set:
        # Vyber uzel s nejnizsim f_score
        f, g, current, path = heapq.heappop(open_set)
        
        # Preskoc, pokud jsme tento uzel uz zpracovali
        if current in closed_set:
            continue
        
        closed_set.add(current)
        explored.append((current, f, g))
        
        # Nasli jsme cil?
        if current == goal:
            return path, explored
        
        # Prozkoumej sousedy
        for dr, dc in directions:
            nr, nc = current[0] + dr, current[1] + dc
            neighbor = (nr, nc)
            
            # Kontrola hranic a zdi
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == 1:  # Zed
                    continue
                if neighbor in closed_set:
                    continue
                
                # Vypocitej nove skore
                new_g = g + 1
                
                # Pokud jsme nasli lepsi cestu
                if neighbor not in g_scores or new_g < g_scores[neighbor]:
                    g_scores[neighbor] = new_g
                    h = manhattan_distance(neighbor, goal)
                    new_f = new_g + h
                    heapq.heappush(open_set, (new_f, new_g, neighbor, path + [neighbor]))
    
    return None, explored  # Cesta neexistuje

print("A* ALGORITMUS")
print("=" * 50)
print("""
Princip:
1. Zacni ve startu
2. Pro kazdy uzel vypocitej f(n) = g(n) + h(n)
3. Vzdy vyber uzel s NEJNIZSIM f(n)
4. Opakuj, dokud nenajdes cil

Kde:
  g(n) = skutecna vzdalenost od startu
  h(n) = odhadovana vzdalenost do cile (heuristika)
  f(n) = celkova odhadovana cena
""")

---

## Prakticky projekt: A* v bludisti

In [ ]:
# Definice bludiste (0 = cesta, 1 = zed)

bludiste = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]

start = (0, 0)
cil = (6, 9)

print("BLUDISTE")
print("=" * 50)
print("\n0 = volna cesta, 1 = zed")
print(f"Start: {start}, Cil: {cil}")
print()

# Zobrazeni bludiste
for r, row in enumerate(bludiste):
    line = ""
    for c, cell in enumerate(row):
        if (r, c) == start:
            line += "[S]"
        elif (r, c) == cil:
            line += "[C]"
        elif cell == 1:
            line += "[#]"
        else:
            line += " . "
    print(line)

In [ ]:
# Spusteni A* algoritmu

print("HLEDANI CESTY POMOCI A*")
print("=" * 50)

cesta, explored = a_star(bludiste, start, cil)

if cesta:
    print(f"\nCesta nalezena! Delka: {len(cesta)-1} kroku")
    print(f"Prozkoumano uzlu: {len(explored)}")
    
    print("\nPostup A* (prvnich 10 kroku):")
    for i, (pos, f, g) in enumerate(explored[:10]):
        h = f - g
        print(f"   {i+1}. {pos}: f={f} (g={g} + h={h})")
    if len(explored) > 10:
        print(f"   ... a dalsich {len(explored)-10} uzlu")
else:
    print("Cesta neexistuje!")

In [ ]:
# Textova vizualizace cesty

print("\nNALEZENA CESTA")
print("=" * 50)

if cesta:
    cesta_set = set(cesta)
    
    print()
    for r, row in enumerate(bludiste):
        line = ""
        for c, cell in enumerate(row):
            if (r, c) == start:
                line += "[S]"
            elif (r, c) == cil:
                line += "[C]"
            elif (r, c) in cesta_set:
                line += " * "
            elif cell == 1:
                line += "[#]"
            else:
                line += " . "
        print(line)
    
    print("\n[S]=Start, [C]=Cil, *=Cesta, #=Zed")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def vizualizuj_astar(grid, path, explored, start, goal):
    """
    Vizualizuje bludiste, cestu a prozkoumane uzly.
    """
    rows = len(grid)
    cols = len(grid[0])
    
    # Vytvor barevnou mapu
    display = np.zeros((rows, cols, 3))
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                display[r, c] = [0.2, 0.2, 0.2]  # Zed - tmave seda
            else:
                display[r, c] = [1, 1, 1]  # Volno - bila
    
    # Oznac prozkoumane uzly (svetle modra)
    explored_set = {pos for pos, _, _ in explored}
    for r in range(rows):
        for c in range(cols):
            if (r, c) in explored_set and (r, c) not in [start, goal]:
                display[r, c] = [0.8, 0.9, 1]  # Svetle modra
    
    # Oznac cestu (oranzova)
    if path:
        for pos in path:
            if pos not in [start, goal]:
                display[pos[0], pos[1]] = [1, 0.6, 0]  # Oranzova
    
    # Start a cil
    display[start[0], start[1]] = [0, 0.8, 0]  # Zelena
    display[goal[0], goal[1]] = [0.8, 0, 0]   # Cervena
    
    # Vykresli
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.imshow(display)
    
    # Pridej mrizku
    ax.set_xticks(np.arange(-0.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, rows, 1), minor=True)
    ax.grid(which='minor', color='gray', linewidth=0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    
    # Pridej f-hodnoty pro prozkoumane uzly
    f_values = {pos: f for pos, f, _ in explored}
    for r in range(rows):
        for c in range(cols):
            if (r, c) in f_values:
                ax.text(c, r, str(f_values[(r, c)]), ha='center', va='center',
                       fontsize=8, color='blue')
    
    ax.set_title(f"A* algoritmus\nZelena=Start, Cervena=Cil, Oranzova=Cesta\n" +
                f"Prozkoumano {len(explored)} uzlu, cesta ma {len(path)-1 if path else 0} kroku")
    plt.tight_layout()
    plt.show()

# Vizualizace
vizualizuj_astar(bludiste, cesta, explored, start, cil)

---

## Porovnani A* s BFS

In [ ]:
from collections import deque

def bfs_grid(grid, start, goal):
    """
    BFS pro porovnani s A*.
    """
    rows = len(grid)
    cols = len(grid[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    queue = deque([(start, [start])])
    visited = {start}
    explored = []
    
    while queue:
        current, path = queue.popleft()
        explored.append(current)
        
        if current == goal:
            return path, explored
        
        for dr, dc in directions:
            nr, nc = current[0] + dr, current[1] + dc
            neighbor = (nr, nc)
            
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == 0 and neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))
    
    return None, explored

# Porovnani
print("POROVNANI A* vs BFS")
print("=" * 50)

# A*
cesta_astar, explored_astar = a_star(bludiste, start, cil)

# BFS
cesta_bfs, explored_bfs = bfs_grid(bludiste, start, cil)

print(f"\n{'Metrika':<25} {'A*':>10} {'BFS':>10}")
print("-" * 47)
print(f"{'Delka cesty':<25} {len(cesta_astar)-1:>10} {len(cesta_bfs)-1:>10}")
print(f"{'Prozkoumanych uzlu':<25} {len(explored_astar):>10} {len(explored_bfs):>10}")

uspory = (1 - len(explored_astar) / len(explored_bfs)) * 100
print(f"\nA* prozkouma o {uspory:.1f}% mene uzlu nez BFS!")
print("\nOba najdou stejne kratkou cestu,")
print("ale A* to udela efektivneji diky heuristice.")

---

## Ruzne heuristiky

In [ ]:
import math

# Ruzne typy heuristik

def manhattan(a, b):
    """Manhattanska vzdalenost - pro 4-smerny pohyb."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def euclidean(a, b):
    """Euklidovska vzdalenost - vzdusna cara."""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def chebyshev(a, b):
    """Chebyshevova vzdalenost - pro 8-smerny pohyb."""
    return max(abs(a[0] - b[0]), abs(a[1] - b[1]))

def zero_heuristic(a, b):
    """Nulova heuristika - A* se chova jako Dijkstra."""
    return 0

# Porovnani heuristik
print("POROVNANI HEURISTIK")
print("=" * 50)

bod_a = (0, 0)
bod_b = (3, 4)

print(f"\nBod A: {bod_a}")
print(f"Bod B: {bod_b}")
print()

heuristiky = [
    ("Manhattan", manhattan),
    ("Euklidovska", euclidean),
    ("Chebyshev", chebyshev),
    ("Nulova", zero_heuristic)
]

for nazev, h in heuristiky:
    hodnota = h(bod_a, bod_b)
    print(f"   {nazev:15}: {hodnota:.2f}")

print("""
Kdy pouzit kterou:
- Manhattan: 4-smerny pohyb (nahoru/dolu/vlevo/vpravo)
- Euklidovska: volny pohyb v prostoru
- Chebyshev: 8-smerny pohyb (vcetne diagonal)
- Nulova: A* se zmeni na Dijkstruv algoritmus
""")

---

## Cviceni

In [ ]:
# CVICENI 1: Vlastni bludiste
# =============================
# Vytvorte vlastni bludiste a najdete cestu

print("CVICENI 1: Vlastni bludiste")
print("=" * 50)

# Vase bludiste (upravte):
moje_bludiste = [
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],
]

muj_start = (0, 0)
muj_cil = (4, 4)

# Test
cesta_m, explored_m = a_star(moje_bludiste, muj_start, muj_cil)

if cesta_m:
    print(f"Cesta nalezena! Delka: {len(cesta_m)-1}")
    print(f"Prozkoumano: {len(explored_m)} uzlu")
    
    # Vizualizace
    cesta_set = set(cesta_m)
    print()
    for r, row in enumerate(moje_bludiste):
        line = ""
        for c, cell in enumerate(row):
            if (r, c) == muj_start:
                line += "[S]"
            elif (r, c) == muj_cil:
                line += "[C]"
            elif (r, c) in cesta_set:
                line += " * "
            elif cell == 1:
                line += "[#]"
            else:
                line += " . "
        print(line)
else:
    print("Cesta neexistuje!")

In [ ]:
# CVICENI 2: Vetsi bludiste
# ==========================
# Vygenerujte nahodne bludiste

import random

print("CVICENI 2: Nahodne bludiste")
print("=" * 50)

def generuj_bludiste(rows, cols, wall_prob=0.3):
    """Vygeneruje nahodne bludiste."""
    grid = []
    for r in range(rows):
        row = []
        for c in range(cols):
            if random.random() < wall_prob:
                row.append(1)  # Zed
            else:
                row.append(0)  # Volno
        grid.append(row)
    
    # Zajisti volny start a cil
    grid[0][0] = 0
    grid[rows-1][cols-1] = 0
    
    return grid

# Generuj a vyres
random.seed(42)  # Pro reprodukovatelnost
nahodne = generuj_bludiste(8, 10, wall_prob=0.25)

start_n = (0, 0)
cil_n = (7, 9)

cesta_n, explored_n = a_star(nahodne, start_n, cil_n)

print(f"\nBludiste 8x10:")
if cesta_n:
    print(f"Cesta nalezena! Delka: {len(cesta_n)-1}")
    cesta_set = set(cesta_n)
    print()
    for r, row in enumerate(nahodne):
        line = ""
        for c, cell in enumerate(row):
            if (r, c) == start_n:
                line += "[S]"
            elif (r, c) == cil_n:
                line += "[C]"
            elif (r, c) in cesta_set:
                line += " * "
            elif cell == 1:
                line += "[#]"
            else:
                line += " . "
        print(line)
else:
    print("Cesta neexistuje - zkuste jiny seed!")

In [ ]:
# CVICENI 3: Porovnani efektivity
# ================================
# Porovnejte A* a BFS na vice bludistich

print("CVICENI 3: Benchmark A* vs BFS")
print("=" * 50)

print("\nGeneruji 5 nahodnych bludisti 15x15...")
print()

vysledky = []

for i in range(5):
    random.seed(i * 10)
    test_grid = generuj_bludiste(15, 15, wall_prob=0.2)
    test_start = (0, 0)
    test_goal = (14, 14)
    
    # A*
    path_a, exp_a = a_star(test_grid, test_start, test_goal)
    
    # BFS
    path_b, exp_b = bfs_grid(test_grid, test_start, test_goal)
    
    if path_a and path_b:
        vysledky.append({
            'astar': len(exp_a),
            'bfs': len(exp_b),
            'path_len': len(path_a) - 1
        })

print(f"{'Bludiste':<10} {'A* uzlu':>10} {'BFS uzlu':>10} {'Uspora':>10}")
print("-" * 45)

for i, v in enumerate(vysledky, 1):
    uspora = (1 - v['astar'] / v['bfs']) * 100
    print(f"{i:<10} {v['astar']:>10} {v['bfs']:>10} {uspora:>9.1f}%")

# Prumer
if vysledky:
    avg_uspora = sum(1 - v['astar']/v['bfs'] for v in vysledky) / len(vysledky) * 100
    print(f"\nPrumerna uspora A* oproti BFS: {avg_uspora:.1f}%")

---

## Kde se A* pouziva?

- **Hry**: NPC hleda cestu k hraci
- **Robotika**: Robot planuje trasu ve skladu
- **GPS navigace**: Hledani optimalni trasy
- **Logistika**: Planovani dorucovacich tras
- **AI ve strategickych hrach**: Jednotky se pohybuji po mape

---

## Shrnuti kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">Co jste se naucili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>* Heuristika = informovany odhad vzdalenosti do cile</li>
        <li>* A* kombinuje: f(n) = g(n) + h(n)</li>
        <li>* g(n) = skutecna vzdalenost od startu</li>
        <li>* h(n) = odhad vzdalenosti do cile</li>
        <li>* A* najde nejkratsi cestu efektivneji nez BFS</li>
        <li>* Manhattan heuristika pro 4-smerny pohyb</li>
    </ul>
    
    <h3 style="color: white;">Klicove dovednosti</h3>
    <p>Umite implementovat A* algoritmus a vybrat spravnou heuristiku.</p>
    
    <h3 style="color: white;">Dalsi kapitola</h3>
    <p>Kapitola 14: Uvod do pravdepodobnosti a Bayesuv teorem</p>
</div>

---

*Notebook pro kurz AI - Blok 2: Algoritmy a hledani*  
*Kompatibilni s Google Colab*